In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.model_selection as model_selection

import time
import datetime
import boto3
import gc
import random
import pickle
import math

from io import StringIO
from collections import defaultdict
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#  Prep

In [ ]:
random.seed(10)

In [ ]:
data_dir = '/Users/Desktop/Classification/'

In [ ]:
US_STATES = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

US_REGIONS = ['MW', 'NE', 'NW', 'SE', 'SW', 'TX', 'UNKNOWN']

CUSTOMER_FACING = ['Y','N']
TRANSIT_OPERATORS =['SINGLE_DRIVER', 'TEAM_DRIVER']

DRIVER_TYPE = ['TEAM','SOLO1','SOLO2']


In [ ]:
loads = pd.read_csv(data_dir + 'loads-Training.txt',sep = "\t" )
len(loads)

In [ ]:
loads.head()

In [ ]:
#len(loads[loads['transit_operator_type'] == 'TEAM_DRIVER'])

In [ ]:
len(loads[loads.average_transit_hour > 690])

In [ ]:
assert(len(loads.load_id.unique()) == len(loads))

In [ ]:
list(loads.columns)

In [ ]:
loads.head().T

In [ ]:
loads = loads[~loads.origin.isnull()]
loads = loads[~loads.final_destination.isnull()]
loads = loads[~loads.origin_zip.isnull()]
loads = loads[~loads.dest_zip.isnull()]
loads = loads[~loads.origin_state.isnull()]
loads = loads[~loads.dest_state.isnull()]

In [ ]:
len(loads)

In [ ]:
def get_lane(row):
    lane = row['origin'] + '->' + row['final_destination']
    return lane
loads['lane'] = loads.apply(get_lane,axis=1)

In [ ]:
loads.count()

In [ ]:
loads.describe()

In [ ]:
loads = loads[loads['checkin_time_windows_at_origin'] >= 0.0]

In [ ]:
len(loads)

In [ ]:
loads.count()

In [ ]:
loads = loads[~loads.creation_date.isnull()]
len(loads)

In [ ]:
loads.describe()

In [ ]:
loads['same_origin'] = np.where(loads['planning_origin'] == loads['origin'],1,0)

loads = loads[loads.same_origin == 1]

len(loads)

In [ ]:
loads['same_dest'] = np.where(loads['planning_dest'] == loads['final_destination'],1,0)

loads = loads[loads.same_dest == 1]

len(loads)

In [ ]:
loads.count()

In [ ]:
loads.describe()

In [ ]:
pd.Series(loads.transit_operator_type.value_counts())

# Read in Blocks

In [ ]:
blocks = pd.read_csv(data_dir + 'Blocks-Training.txt',sep='\t')
blocks.head().T

In [ ]:
len(blocks)

In [ ]:
blocks['startwindowlowerbound'] = pd.to_datetime(blocks['startwindowlowerbound'])
blocks['block_end_utc'] = pd.to_datetime(blocks['block_end_utc'])
blocks['createddate'] = pd.to_datetime(blocks['createddate'])

# Warehouse and Zip Code Match

In [ ]:
loads['origin_zip'] = loads['origin_zip'].astype(str)
loads['origin_zip'] = loads['origin_zip'].apply(lambda x: x.zfill(5))


domiciles_to_main_zip = pd.read_csv(data_dir + 'domiciles_to_main_zip.csv', sep = '\t')

domiciles_to_main_zip['main_zip']  = domiciles_to_main_zip['main_zip'].astype(str)
domiciles_to_main_zip['main_zip'] = domiciles_to_main_zip['main_zip'].apply(lambda x: x.zfill(5))
domiciles_to_main_zip = domiciles_to_main_zip.rename(columns ={'main_zip':'origin_zip'})


print(len(loads))
loads = pd.merge(loads,domiciles_to_main_zip,on = 'origin_zip',how='left')
print(len(loads))

In [ ]:
domiciles_to_main_zip.head()

In [ ]:
loads.domicile = loads.domicile.fillna('missing_domicile')

In [ ]:
domiciles = pd.read_csv(data_dir + 'domicile_to_fc_mapping.csv', sep =',')

def replace_str(row):
    temp = row['all_fullfilment'].replace('[','')
    temp = temp.replace(']','')
    temp = temp.replace('\"','')
    temp = temp.replace('\\','')
    temp  = temp.split(',')
    return temp

domiciles['facilities'] = domiciles.apply(lambda row : replace_str(row),axis=1)
domiciles = domiciles[['domicile_code','facilities']]
domiciles = domiciles.set_index('domicile_code').to_dict()
domiciles = domiciles['facilities'].copy()


domiciles

In [ ]:
fc_domiciles = defaultdict(set) # key: fc values : list of domiciles 

for domicile,fc_list in domiciles.items():
    for fc in fc_list:
        fc_domiciles[fc].add(domicile)

In [ ]:
loads.origin.unique()

In [ ]:
fc_domiciles

In [ ]:
def pd_timestamp_to_int(ts):
    return int(time.mktime(ts.timetuple()))

def int_to_pd_timestamp(t):
    return pd.Timestamp(t, unit='s')

In [ ]:
len(loads)

In [ ]:
loads.head()

In [ ]:
# by lead time
loads['first_checkin_time_utc'] = pd.to_datetime(loads['first_checkin_time_utc'])
def loads_pd_timestamp_to_int(row):
    ts = row['first_checkin_time_utc']
    return int(time.mktime(ts.timetuple()))

loads['creation_date_int'] = loads.apply(lambda row: loads_pd_timestamp_to_int(row),axis = 1)

In [ ]:
cols = ['domicile','startwindowlowerbound','block_end_utc','createddate','blocklength_minutes']   
blocks_to_aggregate = blocks[cols].copy()


block_tuples = [tuple(x) for x in blocks_to_aggregate.to_numpy()]

blocks_dict = defaultdict(list)

for (domicile,start,end,creation,block_length) in block_tuples:
    blocks_dict[domicile].append([pd_timestamp_to_int(start),pd_timestamp_to_int(end),pd_timestamp_to_int(creation),block_length])

new_blocks_dict = {}
for key, values in blocks_dict.items():
    new_blocks_dict[key] =  np.array(values)
blocks_dict = new_blocks_dict

In [ ]:
loads['first_checkin_time_utc'] = pd.to_datetime(loads['first_checkin_time_utc'])
# estimated earliest arrival time back to origin  : first check-in time at origin + average transit hours
loads['earliest_arrival_time'] = loads.apply(lambda row :row['first_checkin_time_utc'] + 2*datetime.timedelta(seconds=(row['average_transit_hour'])*60),axis=1) 

In [ ]:
loads[['first_checkin_time_utc','earliest_arrival_time','average_transit_hour','transit_operator_type']].tail(10)

In [ ]:
cols = ['load_id','origin','first_checkin_time_utc','earliest_arrival_time','creation_date_int','domicile']
loads_tuples =  [tuple(x) for x in loads[cols].to_numpy()]
len(loads_tuples)

In [ ]:
loads.domicile.unique()

In [ ]:
fc_domiciles

In [ ]:
loads_dict = {}
import time
start_time = time.time()

creation_failed = 0
end_failed = 0
start_failed = 0
total = 0

num_loads_without_domicile = 0 
loads_without_domicile = set()

for (l_id,l_o,l_start,l_end,l_p_cycle,l_dom) in loads_tuples:
        
    if l_o not in fc_domiciles and l_dom == 'missing_domicile':
        loads_dict[(l_id,l_p_cycle)] = (0,0)
        num_loads_without_domicile += 1
        loads_without_domicile.add(l_id)

    else:
        num_feasible_blocks = 0
        total_block_duration_length = 0
        
        l_start_int = pd_timestamp_to_int(l_start)
        l_end_int = pd_timestamp_to_int(l_end)
        
        # First check main_zip to domicile
        if l_dom != 'missing_domicile': 
            if l_dom in blocks_dict:
                a = blocks_dict[l_dom]
                temp_a = a[((a[:,0] ) <= (l_start_int)) & ((a[:, 1]) >= (l_end_int)) & (a[:, 2] <= l_p_cycle)] 

                num_feasible_blocks = len(temp_a)
                total_block_duration_length += sum(temp_a[:,3])
            else: 
                if l_o in fc_domiciles: 
                    for domicile in fc_domiciles[l_o]:
                        if domicile in blocks_dict:
                            a = blocks_dict[domicile]

                            temp_a = a[((a[:,0] ) <= (l_start_int)) & ((a[:, 1] ) >= (l_end_int)) & (a[:, 2] <= l_p_cycle)]  
                            num_feasible_blocks = len(temp_a)
                            total_block_duration_length += sum(temp_a[:,3])
                            
        else: 
            if l_o in fc_domiciles: 
                for domicile in fc_domiciles[l_o]:
                    if domicile in blocks_dict:
                        a = blocks_dict[domicile]
                        temp_a = a[((a[:,0]) <= (l_start_int)) & ((a[:, 1]) >= (l_end_int)) & (a[:, 2] <= l_p_cycle)] 
                        num_feasible_blocks = len(temp_a)
                        total_block_duration_length += sum(temp_a[:,3])            
            
                       
        loads_dict[(l_id,l_p_cycle)] = (num_feasible_blocks,total_block_duration_length)
        
    
    if len(loads_dict) % 10000 == 0:
        print(len(loads_dict))
        elapsed_time = time.time() - start_time
        print('done')
        print(elapsed_time)

In [ ]:
num_loads_without_domicile

In [ ]:
df_loads_without_domicile = loads[loads.load_id.isin(loads_without_domicile)]
df_loads_without_domicile.head()

In [ ]:
loads['key'] = loads.apply(lambda row: (row['load_id'],row['creation_date_int']),axis = 1)

df_items = []
for k, v in loads_dict.items():
    df_items.append((k, v[0], v[1]))
loads_dict_df = pd.DataFrame(df_items,columns = ['key','num_feasible_blocks','total_block_minutes'])
loads_dict_df.head()

In [ ]:
loads = pd.merge(loads,loads_dict_df,on = 'key', how='left')
loads.head()

In [ ]:
len(loads)

In [ ]:
missing_blocks_data = loads[loads.num_feasible_blocks == 0]

missing_blocks_data['first_checkin_time_utc'].min()

In [ ]:
pd.Series(loads.num_feasible_blocks).value_counts()

In [ ]:
xx = loads[~loads.load_id.isin(loads_without_domicile)]
print(len(xx))
pd.Series(xx.planning_status_by_blocks.value_counts())

In [ ]:
# For loads we do not know the associated domicile info
loads['has_domicile'] =  np.where(loads['load_id'].isin(loads_without_domicile),0,1)
loads.head().T

In [ ]:
len(loads[loads.has_domicile == 0])

# Set Up Data for Model

In [ ]:
loads.head().T

In [ ]:
(pd.Series(loads.planning_status_by_blocks).value_counts()) 

In [ ]:
loads[~loads.origin_state.isin(US_STATES)]

In [ ]:
loads[~loads.dest_state.isin(US_STATES)]

In [ ]:
loads = loads[loads.dest_state.isin(US_STATES)]
loads = loads[loads.origin_state.isin(US_STATES)]

In [ ]:
len(loads)

In [ ]:
### remove loads that are canceled 
loads = loads[loads.canceled_load == False]
len(loads)

In [ ]:
loads = loads[loads['load_cancellation_date'] == '2050-01-01 00:00:00']
len(loads)

In [ ]:
pd.Series(loads.load_cancellation_date.value_counts())

In [ ]:
pd.Series(loads.num_feasible_blocks).value_counts()

In [ ]:
pd.Series(loads.planning_status_by_blocks).value_counts()

In [ ]:
loads.count()

In [ ]:
loads['is_eventually_unplanned'] = np.where(loads.planning_status_by_blocks == 'Planned in a block',0,1)

In [ ]:
pd.Series(loads['is_eventually_unplanned'].value_counts())

In [ ]:
max(pd.to_datetime(loads.first_checkin_time_utc))

In [ ]:
len(loads)

In [ ]:
loads.describe()

In [ ]:
loads['lead_time_to_departure'] = (pd.to_datetime(loads.first_checkin_time_utc) - pd.to_datetime(loads.creation_date)).astype('timedelta64[h]')

In [ ]:
loads['lead_time_to_departure'].describe()

In [ ]:
print(len(loads))
loads = loads[loads['lead_time_to_departure'] > 0]
print(len(loads))

In [ ]:
loads['lead_time_to_departure'].describe()

In [ ]:
len(loads)

In [ ]:
cols = ['load_id','r4c_adhoc','wims_load','enrichment_flag','rlb_load','planned_load','resource_block_id']
planned_loads = loads[loads.is_eventually_unplanned == 0]
planned_loads[cols].head()

In [ ]:
len(loads)

In [ ]:
loads.describe()

In [ ]:
loads['num_feasible_blocks'].describe()

In [ ]:
len(loads[loads['num_feasible_blocks'] == 0])

In [ ]:
#### Write the output ####
loads.to_csv(data_dir +'teams_w_planned_blocks_total.csv', sep='\t')

In [ ]:
loads.head()

In [ ]:
loads['creation_date'] = pd.to_datetime(loads['creation_date']) 
loads['year'] = loads['creation_date'].dt.year
loads['week'] = loads['creation_date'].dt.isocalendar().week

In [ ]:
loads.head()

In [ ]:
loads2020 = loads[loads.year == 2020]
loads2021 = loads[loads.year == 2021]

In [ ]:
print(len(loads2020))
print(len(loads2021))

In [ ]:
loads1 = loads[(loads.year == 2020) & (loads.week <= 49)]

In [ ]:
loads2 = loads[(loads.year == 2020) & (loads.week <= 50)]

In [ ]:
loads3 = loads[(loads.year == 2020) & (loads.week <= 51)]

In [ ]:
loads4 = loads[(loads.year == 2020) & (loads.week <= 52)]

In [ ]:
loads1.to_csv(data_dir +'train_49.csv', sep='\t')
loads2.to_csv(data_dir +'train_50.csv', sep='\t')
loads2.to_csv(data_dir +'train_51.csv', sep='\t')
loads4.to_csv(data_dir +'train_52.csv', sep='\t')

In [ ]:
loads2020.to_csv(data_dir +'train_2020.csv', sep='\t')
loads2021.to_csv(data_dir +'train_2021.csv', sep='\t')